<h1> Link Health Vending Machine Policy Research </h1>

In [357]:
import pandas as pd
import numpy as np
import requests
import csv
import warnings
warnings.simplefilter('ignore') #Turn off warnings

<h2> Starting Work on Assistance Listings Database </h2>

In [488]:
asstList = pd.read_csv("CSVs/AssistanceListings_DataGov_PUBLIC_WEEKLY_20231021.csv", encoding='unicode_escape')

In [489]:
asstList.head()

,Program Title,Program Number,Popular Name (020),Federal Agency (030),Authorization (040),Objectives (050),Types of Assistance (060),Uses and Use Restrictions (070),Applicant Eligibility (081),Beneficiary Eligibility (082),...,Regional or Local Office (151),Headquarters Office (152),Website Address (153),Related Programs (160),Examples of Funded Projects (170),Criteria for Selecting Proposals (180),Published Date,Parent Shortname,URL,Recovery
0,Agricultural Research Basic and Applied Research,10.001,(Extramural Research),"AGRICULTURAL RESEARCH SERVICE, AGRICULTURE, DE...","{""list"":[{""act"":{""description"":""Food Security ...","To make agricultural research discoveries, eva...",PROJECT GRANTS,Not Applicable,Usually nonprofit institutions of higher educa...,Usually nonprofit institutions of higher educa...,...,"{""flag"":""appendix"",""description"":""See the Agri...","Deborah Watson5601 Sunnyside Avenue, MS 5110, ...",http://www.ars.usda.gov,"10.200 Grants for Agricultural Research, Speci...",Not Applicable.,Peer review.,"Jan 01,1965",USDA,https://sam.gov/fal/8138603977434d65a63073a7e5...,No
1,"Plant and Animal Disease, Pest Control, and An...",10.025,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""act"":{""description"":""Plant Protecti...",To protect U.S. agriculture from economically ...,PROJECT GRANTS,Not Applicable,"Foreign, State, local, and U.S. Territorial go...","Farmers, ranchers, agriculture producers, Stat...",...,"{""flag"":""appendix"",""description"":""Consult Appe...","Eileen M. Berke,4700 River Road, Unit 55, Rive...",http://www.aphis.usda.gov/,10.219 Biotechnology Risk Assessment Research;...,Not Applicable.,Not Applicable.,"Jan 01,1972",USDA,https://sam.gov/fal/47eecdfbe59743e998cf64dafa...,No
2,Wildlife Services,10.028,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""act"":{""description"":""Animal Damage ...",To reduce damage caused by mammals and birds a...,PROJECT GRANTS,Not Applicable,"State and local governments, federally recogni...","States, local jurisdictions, U.S. Territorial ...",...,"{""flag"":""appendix"",""description"":""See Appendix...","Eileen M. Berke,4700 River Road, Unit 55, Suit...",http://www.aphis.usda.gov,15.611 Wildlife Restoration and Basic Hunter E...,Not Applicable.,Relevance to agency program mission and qualif...,"Jan 01,1986",USDA,https://sam.gov/fal/5777f406178e4a95bc9d933a99...,No
3,Indemnity Program,10.030,NaN,"ANIMAL AND PLANT HEALTH INSPECTION SERVICE, AG...","{""list"":[{""act"":{""description"":""Plant Protecti...",Animal and Plant Health Inspection Service adm...,DIRECT PAYMENTS WITH UNRESTRICTED USE,Not Applicable,NaN,NaN,...,"{""flag"":""appendix""}","Donna Cichy100 North 6th Street, Suite 510C\r\...",Not Applicable,Not Applicable.,Not Applicable.,Not Applicable.,"Mar 16,2012",USDA,https://sam.gov/fal/c94b7dc247f64502b368b72013...,No
4,Commodity Loans and Loan Deficiency Payments,10.051,Marketing Assistance Loans (MAL's) and Loan De...,"FARM SERVICE AGENCY, AGRICULTURE, DEPARTMENT OF","{""list"":[{""act"":{""description"":""Agricultural A...","To improve and stabilize farm income, to assis...",DIRECT PAYMENTS WITH UNRESTRICTED USE;DIRECT L...,Not Applicable,"Owner, landlord, tenant, or sharecropper on an...","Owner, landlord, tenant, or sharecropper on a ...",...,"{""flag"":""appendix"",""description"":""Consult the ...","Shayla Watson1400 Independence Avenue, SW, Sto...",http://www.fsa.usda.gov/programs-and-services/...,10.155 Marketing Agreements and Orders;,Not Applicable.,Not Applicable.,"Jan 01,1965",USDA,https://sam.gov/fal/f20d78c22ec34c2d9df3f80ae0...,No


In [490]:
asstList = asstList[["Program Number", 
                     "Program Title", 
                     "Popular Name (020)",
                     "Objectives (050)",
                     "Applicant Eligibility (081)", 
                     "Beneficiary Eligibility (082)", 
                     "Published Date", 
                     "URL"]]

In [491]:
asstList.rename(columns = {"Applicant Eligibility (081)" : "Prerequisites",
                           "Beneficiary Eligibility (082)" : "Beneficiary", 
                           "Popular Name (020)" : "Popular Name", 
                           "Objectives (050)" : "Objectives"}, inplace = True)

In [492]:
asstList['Popular Name'] = asstList['Popular Name'].fillna('')

In [493]:
def programNameMerge(row):
    if row['Popular Name'] != '' and row['Popular Name'] != row["Program Title"]:
        return f"{row['Program Title']} or {row['Popular Name']}"
    else:
        return row['Program Title']

asstList["Program Title"] = asstList.apply(programNameMerge, axis=1)
asstList = asstList.drop("Popular Name", axis = 1)

<h2> Find Relevant Programs in Assistance Listing DB </h2>

<p> Organized the columns that we need to find prerequisites, now need to find a way to extract all relevant programs (i.e. ones for individuals rather than orgs.) </p>

In [494]:
# Want to keep all programs that have prereqs listed.
asstList = asstList[asstList["Prerequisites"].notnull()]
"Number of Programs with Prerequisites: " + str(len(asstList))

'Number of Programs with Prerequisites: 2371'

In [495]:
# ACP Eligibility Requirements...
asstList[asstList["Program Title"] == "Affordable Connectivity Program"]["Prerequisites"].iloc[0]
# do all programs use ';' to split each prerequistes?

'Provider Eligibility: In the Infrastructure Act, a \x93participating provider\x94 for the Affordable Connectivity Program shall be a broadband service provider that is either designated as an eligible telecommunications carrier (ETC) or seeks approval from the Commission for participation in the Affordable Connectivity Program. ETCs and non-ETCs must establish they provide broadband service to participate in the program. Household Eligibility: Support is available only for one line per household; applicants are eligible if at least one member of the household: Income is at or below 200% of the federal poverty guidelines; Meets the qualifications for participation in the Lifeline program, that is, if they participate in one of the following federal assistance programs: Medicaid, Supplemental Nutrition Assistance Program, Supplemental Security Income, Federal Public Housing Assistance, or Veterans Pension and Survivors Benefit; or one of the following Tribal-specific federal assistance 

In [539]:
wordsToAvoid = "nonprofit|institutions|organizations|cooperatives|pvo|ngo"
wordsWeWant = "individuals|applicants|household|person|families"
indPrograms = asstList[asstList["Beneficiary"].str.lower().str.contains(wordsWeWant) == True 
                       & asstList["Beneficiary"].str.lower().str.contains(wordsToAvoid) == False]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [497]:
# Formatting the dates in indPrograms.
monthDict = {"Jan" : "01", "Feb" : "02", "Mar" : "03", "Apr" : "04", "May" : "05", "Jun" : "06", "Jul" : "07", 
             "Aug" : "08", "Sep" : "09", "Oct" : "10", "Nov" : "11", "Dec" : "12"}

def formatDate(row):
    date = f"{row['Published Date']}"
    month = monthDict[date[:3]]
    day = date[4:6]
    year = date[7:11]
    return year + "-" + month + "-" + day

indPrograms["Published Date"] = indPrograms.apply(formatDate, axis = 1)
indPrograms.sort_values("Published Date")

,Program Number,Program Title,Objectives,Prerequisites,Beneficiary,Published Date,URL
206,10.555,National School Lunch Program or School Lunch,"To assist States, through cash grants and food...","States, including the District of Columbia and...",All children enrolled in schools where this pr...,1965-01-01,https://sam.gov/fal/bf821e9376ce450e91ec88e172...
567,14.128,Mortgage Insurance Hospitals or (Section 242 -...,To facilitate the affordable financing of hosp...,Qualified applicants can be either profit or n...,Persons needing the services of these hospital...,1965-01-01,https://sam.gov/fal/8b065c2bc23f43029200c24cb4...
569,14.133,Mortgage Insurance Purchase of Units in Condom...,To enable individuals and families to purchase...,All legal residents are eligible to apply.,Individuals/families.,1965-01-01,https://sam.gov/fal/329d2eca2ba04785a8eef2b26c...
145,10.405,Farm Labor Housing Loans and Grants or (Labor ...,"To provide decent, safe, and sanitary low-rent...","Loans are available to farmers, family farm pa...",A domestic farm laborer is any person who rece...,1965-01-01,https://sam.gov/fal/f6a2b763b3a44bff865e50e90a...
146,10.406,Farm Operating Loans and Loan Guarantees or Fa...,The Farm Service Agency (FSA) offers direct an...,General Eligibility for Direct Loans\n\nThe fo...,"Individual/Entity, Socially Disadvantaged Appl...",1965-01-01,https://sam.gov/fal/75a2b880bdef42229df6b78d96...
...,...,...,...,...,...,...,...
1841,90.705,Southeast Crescent Regional Commission - Econ...,The mission of the Commission is to help build...,"1.)\tState governments of Alabama, Florida, Ge...",Funds awarded under this listing must benefit ...,2023-03-03,https://sam.gov/fal/5c16d3e634984e67b4b83993e4...
143,10.384,Indigenous Animals Harvesting and Meat Process...,This program will provide funds to Tribal gove...,Not Applicable,Tribal governments as defined by the Federally...,2023-03-29,https://sam.gov/fal/7dc6d10cc7594f3cbd218e3156...
1659,66.965,Alaska Native Claims Settlement Act Contaminat...,EPA is establishing and implementing a program...,"Alaska Native Regional Corporations, Alaskan N...","Alaska Native Regional Corporations, Alaskan N...",2023-04-04,https://sam.gov/fal/040fea5cb4584ab098f2fc7bad...
1515,64.058,Veteran and Spouse Transitional Assistance Gra...,The Secretary of Veterans Affairs will make gr...,The following organizations and entities are e...,Veteran/Service person/Reservist (including de...,2023-04-20,https://sam.gov/fal/2e3c4cc68026412bb1ee8748c9...


In [498]:
# Sorting programs in ascending order by date. 
indPrograms = indPrograms.sort_values("Published Date", ascending = True)
# If no date set to oldest date...
# indPrograms = indPrograms[indPrograms["Prerequisites"] != "Not Applicable"]

<h2> Extracting individual Prerequistes from Prerequiste Text Block </h2>

In [502]:
prereqsSplit = indPrograms["Prerequisites"]

In [503]:
prereqsSplit[prereqsSplit.str.contains("requirements")].iloc[5]

'FORMULA grants: States, Territories and Tribes as defined in the 54 USC 301 et seq (commonly known as the National Historic Preservation Act), operate programs administered by a State Historic Preservation Officer or Tribal Historic Preservation Officer appointed by the Governor, Tribal government or according to State or Tribal law, and which are otherwise in compliance with the requirements of the Act. \r\n\r\n\r\nPROJECT grants: Eligible tribal applicants include Federally recognized Indian Tribes, Alaska Native Corporations, and Native Hawaiian organizations.\r\n\r\nOther project grants follow eligibility as defined  by Congress and stated in statue or committee language.'

In [536]:
#removeTest = ['\n', '\t', 'I.', 'II.', 'III.', 'IV.', "V.", "VI.", 'VII.', 'VIII.', 'IX.', 'X.']

remove = ['\n', '\t']
newSplit = prereqsSplit
for char in remove:
    newSplit = newSplit.str.replace(char, '')
newSplit = newSplit.drop(2228).str.split(';', expand = True)

# droppign 2228 bc has like 50 splits, all useless..
# newSplit = prereqsSplit.drop(2228).str.replace('\n', '').str.replace('\t', '').str.split(';', expand = True)
newSplit

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
206,"States, including the District of Columbia and...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
567,Qualified applicants can be either profit or n...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
569,All legal residents are eligible to apply.,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
145,"Loans are available to farmers, family farm pa...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
146,General Eligibility for Direct LoansThe follow...,II.Have a satisfactory credit history,"III.Be a citizen of the United States, includi...",a U.S. non- citizen national or a qualified a...,IV.Be unable to obtain credit elsewhere at rea...,V.Have the legal capacity to incur the obligat...,VI. Not have outstanding unpaid judgments obta...,VII.Not be delinquent on a federal debt,VIII.Must not have provided FSA with false or ...,IX.Not have been convicted under federal or st...,...,"or any occasion after April 4, 1996VIII.Be th...",XI.Not have provided FSA with false or mislead...,XII. Not have been convicted under federal or ...,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,"State (includes District of Columbia, public i...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1841,"1.)State governments of Alabama, Florida, Geor...",§ 200.1 Federally recognized Indian Tribe - ...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1659,"Alaska Native Regional Corporations, Alaskan N...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1515,The following organizations and entities are e...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [537]:
# need to find a way to formulate questions from each given question...